# Convolutional Neural Networks - Image Classification

### 1. INITIALIZATION

In [1]:
# Import necessary libraries and modules
import pandas as pd

#https://www.analyticsvidhya.com/blog/2020/02/learn-image-classification-cnn-convolutional-neural-networks-3-datasets/

### 2. LOADING DATASET

The dataset used is CIFAR-10, it contains 60k (50k training and 10k testing) coloured images with 10 classes.

In [2]:
# Loading the dataset from keras library
from keras.datasets import cifar10 
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 22s 0us/step
